### Resources

In [25]:
import argparse
import numpy as np
import sys

from collections import Counter
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix, lil_matrix, vstack
from scipy.stats import gamma

### Functions

In [26]:
# Step 1
# Read in vocabulary from file.
def get_vocab(vocab_fn, ignore_case):
    vocab = []
    vocab_index = {}
    for i, line in enumerate(open(vocab_fn, mode='r', encoding='utf-8')):
        term = line.strip()
        if ignore_case:
            term = term.lower()
        vocab.append(term)
        vocab_index[term] = i
    return vocab, vocab_index

In [27]:
# Step 2
# From input corpus in_tsv and the index of working vocabulary vocab_index
# construct:
#   authors: working list of authors
#   author_doc_ids: mapping of authors to document ids
#   doc_term_matrix: document-term matrix
def process_corpus(in_tsv, vocab_index, ignore_case, verbose):
    vocab_size = len(vocab_index)
    authors_by_doc = []
    doc_vectors = []
    n_lines = sum(1 for line in open(in_tsv))
    reader = open(in_tsv, mode='r', encoding='utf-8')
    for i, line in enumerate(reader):
        if verbose and i and i % 1000 == 0:
            print('{}/{}'.format(i, n_lines), file=sys.stderr)
        fields = line.strip().split('\t')
        authors_by_doc.append(fields[1])
        vector = lil_matrix((1, vocab_size))
        tokens = fields[2].split()
        if ignore_case:
            tokens = [t.lower() for t in tokens]
        term_counts = Counter(tokens)
        for term in term_counts:
            if term in vocab_index:
                col = vocab_index[term]
                vector[0, col] = term_counts[term]
        doc_vectors.append(vector)
    doc_term_matrix = vstack(doc_vectors, format='csr')
    authors = sorted(list(set(authors_by_doc)))
    author_index = {author: i for i, author in enumerate(authors)}
    author_doc_ids = {author: [] for author in authors}
    for i, a in enumerate(authors_by_doc):
        author_doc_ids[a].append(i)
    return authors, author_index, author_doc_ids, doc_term_matrix

In [28]:
# Step 3
# Construct author-term matrix from document-term matrix.
def get_author_term_matrix(authors, author_doc_ids, doc_term_matrix):
    author_vectors = [csr_matrix(doc_term_matrix[doc_ids].sum(axis=0)) for
                      doc_ids in author_doc_ids.values()]
    author_term_matrix = vstack(author_vectors, format='csc')
    return author_term_matrix

In [29]:
# Used in Step 4

# Estimate gamma parameters k, theta using method of moments
def get_gamma_parameters(author_term_freqs):
    term_means = np.mean(author_term_freqs, axis=0).getA1()
    author_term_freqs = author_term_freqs.toarray() # Added to resolve error
    term_vars = np.var(author_term_freqs, axis=0, ddof=1)#.getA1() Dropped this to resolve error
    ks = np.divide(np.square(term_means), term_vars)
    thetas = np.divide(term_vars, term_means)
    return ks, thetas

In [30]:
# Step 4
# Build author-term stop weight matrix for given author-term matrix and
# probability threshold.
def get_stop_weights(author_term_matrix, threshold):
    n_authors, n_terms = author_term_matrix.shape
    weights = lil_matrix((n_authors, n_terms))
    author_term_freqs = author_term_matrix / author_term_matrix.sum(axis=1)
    ks, thetas = get_gamma_parameters(author_term_freqs)

    # Assign probabilities of deletion for each partition-feature pair
    author_term_freqs = csc_matrix(author_term_freqs) # Added to convert non-indexable COO to indexable CSC format 
    
    for term_id in range(n_terms):
        author_freqs = author_term_freqs[:, term_id].toarray().flatten() # Added to replace the line below
        #author_freqs = author_term_freqs[:, term_id].getA1() # Replaced wth line above
        freq_threshold = gamma.ppf(1-threshold, ks[term_id],
                                   scale=thetas[term_id])
        for author_id, freq in enumerate(author_freqs):
            if freq_threshold < freq:
                weights[author_id, term_id] = 1 - freq_threshold / freq
    weights = weights.tocsr()
    return weights


In [31]:
# Step 5
# Downsample input according to the author-term stop weights matrix.
def downsample(in_tsv, vocab_index, doc_term_matrix, author_index, # changed from document_term_matrix
               weights, out_tsv, min_tokens, ignore_case, verbose): # changed from author_term_weights
    writer = open(out_tsv, mode='w', encoding='utf-8')
    n_docs = doc_term_matrix.shape[0] # Changed from document_term_matrix
    for doc_id, line in enumerate(open(in_tsv, mode='r', encoding='utf-8')):
        if verbose and doc_id and doc_id % 1000 == 0:
            print('{}/{}'.format(doc_id, n_docs), file=sys.stderr)
        fields = line.strip().split('\t')
        author = fields[1]
        author_id = author_index[author]
        tokens = fields[2].split()
        # Filter tokens based on working vocabulary
        if ignore_case:
            tokens = [t for t in tokens if t.lower() in vocab_index]
            term_ids = np.array([vocab_index[t.lower()] for t in tokens])
        else:
            tokens = [t for t in tokens if t in vocab_index]
            term_ids = np.array([vocab_index[t] for t in tokens])

        # Construct token mask for curation
        term_stop_rates = weights.getrow(author_id) # changed from author_term_weights
        term_stop_rates = term_stop_rates.toarray().ravel()
        token_keep_rates = [1-term_stop_rates[i] for i in term_ids]
        token_mask = np.random.binomial(1, token_keep_rates)
        n_kept = token_mask.sum()

        # Write document if it has at least min_tokens tokens
        if n_kept >= min_tokens:
            token_mask = token_mask.astype(bool)
            stopped_text = ' '.join(np.array(tokens)[token_mask])
            writer.write('{}\t{}\t{}\n'.format(fields[0], fields[1],
                                               stopped_text))
    writer.close()

### Execution

In [32]:
# Step 1
vocab = get_vocab("20240909_PhD_DiaryDictionary.txt", ignore_case=True)
vocab_index = vocab[1]
#vocab_index

In [33]:
# Step 2
corpus = process_corpus("20240909_PhD_DiaryCorpus.tsv", vocab_index, ignore_case=True, verbose=False)
authors = corpus[0]
author_index = corpus[1]
author_doc_ids = corpus[2]
doc_term_matrix = corpus[3]

In [34]:
# Step 3
author_term_matrix = get_author_term_matrix(authors, author_doc_ids, doc_term_matrix)
#print(author_term_matrix)
#author_term_matrix

In [41]:
# Step 4
weights = get_stop_weights(author_term_matrix, threshold=0.1)

In [42]:
# Step 4
downsample("20240909_PhD_DiaryCorpus.tsv", vocab_index, doc_term_matrix, author_index,
               weights, "20240911_PhD_DiaryCorpus.tsv", min_tokens=1, ignore_case=True, verbose=False)

## Troubleshooting

The get_stop_weights function produces a sparse matrix (COO) called author_term_freqs, which the get_gamma_parameters function calls to calculate variance; however, SciPy does not have a variance function for sparse matrix. Trying out different ways to solve. Options include: 1) converting the sparse matrix to a numpy array, then running just the variance on that; 2) doing the same but before calculating the means, given the advice in the Scipy documentation to convert before applying numpy calculations; 3) using a custom function to calculate variance directly on the sparse arrray. ChatGPT was used to explore possible solutions to this problem. The variance function was obtained from https://gist.github.com/sumartoyo/edba2eee645457a98fdf046e1b4297e4. The following Scipy and Numpy documentation were also used. Solution 1 and 2 produced the same results while solution 3 produced different figures as shown below. This might be because the function could not accommodate the degrees of freedom parameter. To minimize the amount of meddling with the original code, I will stick with solution 1.
<ul>
    <li>https://docs.scipy.org/doc/scipy-1.12.0/reference/sparse.html#sparse-matrix-classes</li>
<li>https://docs.scipy.org/doc/scipy-1.12.0/reference/generated/scipy.sparse.coo_matrix.mean.html#scipy.sparse.coo_matrix.mean</li>
    <li>https://numpy.org/devdocs/reference/generated/numpy.matrix.mean.html#numpy.matrix.mean</li>
    </ul>

In [130]:
#n_authors, n_terms = author_term_matrix.shape
#weights = lil_matrix((n_authors, n_terms))
#author_term_freqs = author_term_matrix / author_term_matrix.sum(axis=1)
#ks, thetas = get_gamma_parameters(author_term_freqs)

In [135]:
# Solution 1

#def get_gamma_parameters(author_term_freqs):
#    term_means = np.mean(author_term_freqs, axis=0).getA1()
#    author_term_freqs = author_term_freqs.toarray()
#    term_vars = np.var(author_term_freqs, axis=0, ddof=1)
#    ks = np.divide(np.square(term_means), term_vars)
#    thetas = np.divide(term_vars, term_means)
#    return ks, thetas

#n_authors, n_terms = author_term_matrix.shape
#weights = lil_matrix((n_authors, n_terms))
#author_term_freqs = author_term_matrix / author_term_matrix.sum(axis=1)
#ks, thetas = get_gamma_parameters(author_term_freqs)

#print(ks, thetas)

[0.25 0.25 0.25 ... 0.25 0.25 0.25] [1.46563095e-04 5.80214679e-05 5.80214679e-05 ... 5.80214679e-05
 5.80214679e-05 3.68935621e-04]


In [133]:
# Solution 2

#def get_gamma_parameters(author_term_freqs):
#    author_term_freqs = author_term_freqs.toarray()
#    term_means = np.mean(author_term_freqs, axis=0)
#    term_vars = np.var(author_term_freqs, axis=0, ddof=1)
#    ks = np.divide(np.square(term_means), term_vars)
#    thetas = np.divide(term_vars, term_means)
#    return ks, thetas

#n_authors, n_terms = author_term_matrix.shape
#weights = lil_matrix((n_authors, n_terms))
#author_term_freqs = author_term_matrix / author_term_matrix.sum(axis=1)
#ks, thetas = get_gamma_parameters(author_term_freqs)

#print(ks, thetas)

[0.25 0.25 0.25 ... 0.25 0.25 0.25] [1.46563095e-04 5.80214679e-05 5.80214679e-05 ... 5.80214679e-05
 5.80214679e-05 3.68935621e-04]


In [136]:
# Solution 3

#def get_gamma_parameters(author_term_freqs):
#    term_means = np.mean(author_term_freqs, axis=0).getA1()
#    term_vars = vars(author_term_freqs, axis=0).getA1()
#    ks = np.divide(np.square(term_means), term_vars)
#    thetas = np.divide(term_vars, term_means)
#    return ks, thetas

#def vars(a, axis=0):
#    """ Variance of sparse matrix a
#    var = mean(a**2) - mean(a)**2
#    """
#    a_squared = a.copy()
#    a_squared.data **= 2
#    return a_squared.mean(axis) - np.square(a.mean(axis))

#n_authors, n_terms = author_term_matrix.shape
#weights = lil_matrix((n_authors, n_terms))
#author_term_freqs = author_term_matrix / author_term_matrix.sum(axis=1)
#ks, thetas = get_gamma_parameters(author_term_freqs)

#print(ks, thetas)

[0.33333333 0.33333333 0.33333333 ... 0.33333333 0.33333333 0.33333333] [1.09922322e-04 4.35161010e-05 4.35161010e-05 ... 4.35161010e-05
 4.35161010e-05 2.76701716e-04]


Another hitch with the get stop weights function occurred with the creation of the author_freqs object, which tried to index over a coo_matrix, which cannot be done. When I converted this to csc_matric, the getA1() method did not work. I used ChatGPT to help me resolve this problem but the result yields an empty matrix. This might be because the threshhold is too low. Gradually increased until the matrix was no longer empty. 

In [96]:
n_authors, n_terms = author_term_matrix.shape
weights = lil_matrix((n_authors, n_terms))
author_term_freqs = author_term_matrix / author_term_matrix.sum(axis=1)
ks, thetas = get_gamma_parameters(author_term_freqs)

# Assign probabilities of deletion for each partition-feature pair
author_term_freqs = csc_matrix(author_term_freqs) # Added to convert non-indexable COO to indexable CSC format 
    
for term_id in range(n_terms):
    author_freqs = author_term_freqs[:, term_id].toarray().flatten() # Added to replace the line below
    #author_freqs = author_term_freqs[:, term_id].getA1() # Replaced wth line above
    freq_threshold = gamma.ppf(1-.07, ks[term_id],
                                   scale=thetas[term_id])
    for author_id, freq in enumerate(author_freqs):
        if freq_threshold < freq:
            weights[author_id, term_id] = 1 - freq_threshold / freq
weights = weights.tocsr()
weights

<4x4887 sparse matrix of type '<class 'numpy.float64'>'
	with 3842 stored elements in Compressed Sparse Row format>

In [102]:
# Diagnostics

#author_freqs
weights.getrow(3) # Non-zero values -- Housewife (1568), Labourer (218), Businessman (1152), Lady (904)
#print(author_term_freqs.getcol(0))
#author_term_freqs.shape
#type(weights)
#weights.get_shape()
#print(weights.getrow(0))
#weights.getcol(0)
#weights.count_nonzero()
#print(weights)
#author_term_matrix # CSC 4x4887, 6521
#n_authors, n_terms # (4, 4887)
#weights # List of Lists format 0 to CSR 0
#author_term_freqs # COO 4x4887, 6521 to CSC 4x4887, 6521 (alt CSR 4x4887, 6521)
#len(ks) # 4887
#len(thetas) # 4887
#len(author_freqs) # 4
#freq_threshold # 0.00044645494832863414

<1x4887 sparse matrix of type '<class 'numpy.float64'>'
	with 904 stored elements in Compressed Sparse Row format>